In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format
warnings.filterwarnings("ignore")

url_daily = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/daily_peak_transformers_dataset.csv"
df_daily = pd.read_csv(url_daily,  sep=';', encoding='latin-1')

df = df_daily

In [5]:
import pandas as pd
import numpy as np
import os
import warnings
import matplotlib.pyplot as plt # Importa o submódulo para plotagem
import matplotlib # <--- CORREÇÃO: Importa o pacote base Matplotlib
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

# Configurações para reprodutibilidade e ambiente
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings("ignore")
plt.rcParams['figure.figsize'] = (14, 6)
matplotlib.use('Agg') # Agora esta linha deve funcionar


# ========================================================================
# 1. SIMULAÇÃO E PREPARAÇÃO DO DATAFRAME (FOCANDO NO T1)
# ========================================================================

# Carregamento dos dados
url_daily = "https://media.githubusercontent.com/media/ruanvirginio/masters/refs/heads/main/bases_tratadas/daily_peak_transformers_dataset.csv"
df = pd.read_csv(url_daily, sep=';', encoding='latin-1')
df['datahora'] = pd.to_datetime(df['datahora'])


def create_simulated_df(df_input):
    # Focar apenas no T1 e garantir que 'datahora' seja datetime
    df_t1 = df_input[df_input['id'] == 'T21a'].copy()
    df_t1 = df_t1.sort_values('datahora').dropna()
    
    # Se a filtragem não retornar dados (ex: se o T1 não estiver no seu df real), 
    # usa uma série temporal simulada para a demonstração.
    if df_t1.empty or len(df_t1) < 2500:
        print("AVISO: Usando série temporal simulada (T1) para demonstração.")
        start_date = '2018-01-01'
        end_date = '2024-12-31'
        dates = pd.date_range(start=start_date, end=end_date, freq='D')
        
        time = np.arange(len(dates))
        trend = time / 3000
        yearly_seasonality = 0.2 * np.sin(2 * np.pi * time / 365.25)
        weekly_seasonality = 0.1 * np.sin(2 * np.pi * time / 7)
        noise = np.random.normal(0, 0.05, len(dates))
        
        S_values = 0.5 + trend + yearly_seasonality + weekly_seasonality + noise
        S_values = np.clip(S_values, 0, 1) # Garante escala 0-1
        
        df_t1 = pd.DataFrame({'datahora': dates, 'S': S_values, 'id': 'T1'})
    
    return df_t1

df_t1 = create_simulated_df(df)
S_values = df_t1['S'].values
print(f"Dados do T1 prontos. Total de amostras: {len(S_values)}")


# ========================================================================
# 2. FUNÇÕES DE DATA PREPARATION E MODELAGEM
# ========================================================================

def create_multi_output_dataset(data, janela, horizonte):
    """ Cria os conjuntos X (input window) e y (multi-output horizon) """
    
    n_samples = len(data) - janela - horizonte + 1
    if n_samples <= 0:
        return None, None, None
    
    # X: Janela de histórico (shape: n_samples, janela)
    X = np.lib.stride_tricks.sliding_window_view(data, janela)[:n_samples]
    
    # y: Horizonte de previsão (shape: n_samples, horizonte)
    y = np.lib.stride_tricks.sliding_window_view(data[janela:], horizonte)[:n_samples]
    
    # Ajuste para formato 3D (n_samples, janela, 1) necessário para LSTM
    X_lstm = X.reshape((X.shape[0], X.shape[1], 1))
    
    return X_lstm, y, n_samples


def build_lstm_model(janela, horizonte, units=50):
    """ Constrói e compila o modelo LSTM Multi-Output """
    model = Sequential()
    # Usando relu para evitar o problema do vanishing gradient em redes rasas
    model.add(LSTM(units=units, activation='relu', input_shape=(janela, 1))) 
    # Camada de saída com 'horizonte' unidades (Multi-Output)
    model.add(Dense(horizonte))
    
    model.compile(optimizer='adam', loss='mse')
    return model


# ========================================================================
# 3. EXECUÇÃO PARA MÚLTIPLOS HORIZONTES
# ========================================================================

HORIZONTES = {
    '3 Meses': 90,
    '6 Meses': 180,
    '1 Ano': 365,
    '3 Anos': 1095
}
JANELA_HISTORICO = 30 # Janela de 30 dias de histórico
TEST_SIZE = 0.2       # 20% para teste

resultados = []

for nome, H in HORIZONTES.items():
    print(f"\n--- Treinando para Horizonte: {nome} (H={H} dias) ---")
    
    # 3.1. Preparação dos dados para o horizonte H
    X, y, n_samples = create_multi_output_dataset(S_values, JANELA_HISTORICO, H)
    
    if n_samples is None or n_samples < 50: # Mínimo de amostras para treino
        print(f"ERRO: Dados insuficientes para H={H}. Pulando.")
        continue

    # 3.2. Split (80% treino, 20% teste) - Mantém a ordem temporal
    split_point = int((1 - TEST_SIZE) * n_samples)
    
    X_train, X_test = X[:split_point], X[split_point:]
    y_train, y_test = y[:split_point], y[split_point:]
    
    print(f"Split: Treino={len(X_train)} amostras, Teste={len(X_test)} amostras.")
    
    # 3.3. Construção e Treinamento do Modelo
    model = build_lstm_model(JANELA_HISTORICO, H, units=50)
    
    # Treino
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)
    
    # 3.4. Previsão e Avaliação
    y_pred = model.predict(X_test, verbose=0)
    
    # Calculando métricas (RMSE e MAE sobre TODO o horizonte H)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    # Calculando métricas para o PRIMEIRO PASSO (H=1) para comparação justa
    rmse_h1 = sqrt(mean_squared_error(y_test[:, 0], y_pred[:, 0]))
    mae_h1 = mean_absolute_error(y_test[:, 0], y_pred[:, 0])

    print(f"Métricas (Média sobre H={H}): RMSE={rmse:.4f}, MAE={mae:.4f}")
    print(f"Métricas (Apenas H=1): RMSE={rmse_h1:.4f}, MAE={mae_h1:.4f}")
    
    resultados.append({
        'Horizonte': nome,
        'Dias (H)': H,
        'RMSE Médio (H)': np.round(rmse, 4),
        'MAE Médio (H)': np.round(mae, 4),
        'RMSE H=1': np.round(rmse_h1, 4),
        'MAE H=1': np.round(mae_h1, 4)
    })

# ========================================================================
# 4. RESULTADOS E PLOTAGEM
# ========================================================================

resultados_df = pd.DataFrame(resultados)
print("\n" + "="*50)
print("RESUMO DE ACURÁCIA POR HORIZONTE (LSTM T1)")
print("="*50)
print(resultados_df.set_index('Horizonte'))

# Plotagem de H=1 de cada Horizonte (Comparação)
def plot_h1_comparison(df_history, y_test, y_pred, H, nome, split_point, JANELA_HISTORICO):
    
    # Datas correspondentes ao teste
    dates_start_index = split_point + JANELA_HISTORICO
    dates_end_index = dates_start_index + len(y_test)
    dates_test = df_history['datahora'].iloc[dates_start_index : dates_end_index]
    
    plt.figure()
    plt.plot(dates_test, y_test[:, 0], label='Real (H=1)', color='blue')
    plt.plot(dates_test, y_pred[:, 0], label=f'Previsto (H=1 de H={H})', linestyle='--', color='orange')
    
    plt.title(f'Previsão de Teste (H=1) para Horizonte {nome}')
    plt.xlabel('Data')
    plt.ylabel('Potência Aparente (Escala 0-1)')
    plt.legend()
    plt.tight_layout()
    os.makedirs('plots_horizontes', exist_ok=True)
    
    # Usa Matplotlib para salvar sem Kaleido
    plt.savefig(f'plots_horizontes/LSTM_H1_de_{H}dias.pdf', dpi=300) 
    plt.close()

# Executa o plot para todos os horizontes (para fins de relatório)
for nome, H in HORIZONTES.items():
    X, y, n_samples = create_multi_output_dataset(S_values, JANELA_HISTORICO, H)
    if n_samples is not None and n_samples >= 50:
        split_point = int((1 - TEST_SIZE) * n_samples)
        X_test, y_test = X[split_point:], y[split_point:]
        
        # Treina o modelo novamente (ou carregue o modelo treinado se disponível)
        model = build_lstm_model(JANELA_HISTORICO, H, units=50)
        model.fit(X[:split_point], y[:split_point], epochs=20, batch_size=32, verbose=0)
        y_pred = model.predict(X_test, verbose=0)
        
        plot_h1_comparison(df_t1, y_test, y_pred, H, nome, split_point, JANELA_HISTORICO)

print("\nOs gráficos comparando a previsão H=1 de cada modelo foram salvos na pasta 'plots_horizontes'.")
print("\nA tendência é que o RMSE Médio (H) aumente drasticamente com o horizonte, confirmando a dificuldade de prever o Longo Prazo.")

Dados do T1 prontos. Total de amostras: 2513

--- Treinando para Horizonte: 3 Meses (H=90 dias) ---
Split: Treino=1915 amostras, Teste=479 amostras.
Métricas (Média sobre H=90): RMSE=0.1599, MAE=0.1357
Métricas (Apenas H=1): RMSE=0.1594, MAE=0.1403

--- Treinando para Horizonte: 6 Meses (H=180 dias) ---
Split: Treino=1843 amostras, Teste=461 amostras.
Métricas (Média sobre H=180): RMSE=0.1625, MAE=0.1370
Métricas (Apenas H=1): RMSE=0.1637, MAE=0.1403

--- Treinando para Horizonte: 1 Ano (H=365 dias) ---
Split: Treino=1695 amostras, Teste=424 amostras.
Métricas (Média sobre H=365): RMSE=0.1641, MAE=0.1425
Métricas (Apenas H=1): RMSE=0.1554, MAE=0.1363

--- Treinando para Horizonte: 3 Anos (H=1095 dias) ---
Split: Treino=1111 amostras, Teste=278 amostras.
Métricas (Média sobre H=1095): RMSE=0.1619, MAE=0.1392
Métricas (Apenas H=1): RMSE=0.1588, MAE=0.1338

RESUMO DE ACURÁCIA POR HORIZONTE (LSTM T1)
           Dias (H)  RMSE Médio (H)  MAE Médio (H)  RMSE H=1  MAE H=1
Horizonte           